In [1]:
import llm_core.llm as L
import os

In [2]:
#os.environ['MODEL_PATH'] = "/data/ai_club/llms/qwen2-7b-instruct-q5_k_m.gguf"
LLM_GLOBAL_INSTANCE = L.Llama(
                n_ctx=1000,
                model_path=os.environ['MODEL_PATH'],
                n_gpu_layers=-1, verbose=0,
                # embedding=True
            )

In [3]:
# system_input = "You are a Japanese language assistant who speaks in consice, simple sentences."
# user_input = "Tell me about your favorite Kanji, in Japanese using Kanji, Hiragana, and Katakana when appropriate"
system_input = "You are a helpful assistant. Respond with prompts suitable for Japanese children's books, following the [word] が [word] です form."
user_input = "赤色は何をしてるですか？"

In [4]:
# No Kanji usage, rare katakana usage
grammar_string = r"""
root ::= sentence{1,2}
allowedchars ::= [\u3040-\u30FF, \u4e00 - \u9faf]
word ::= allowedchars
sentence ::= word particle word desu end
particle ::= "が"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], grammar=my_grammar)

{'id': 'chatcmpl-7809e046-be9a-483d-86c3-d3d350a45373',
 'object': 'chat.completion',
 'created': 1739716189,
 'model': '/data/ai_club/llms/qwen2.5-7b-instruct-q8_0-00001-of-00003.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant', 'content': 'あがめです。'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 50, 'completion_tokens': 5, 'total_tokens': 55}}

In [5]:
#With basic prompt, no character restrictions spamps \u0000 or \uffff
#Good responses with user_input = "Tell me about your favorite Kanji, in Japanese using Kanji, Hiragana, and Katakana when appropriate"
#Talks in any vocab, but keeps sentences in が + です format. :D
# Try telling it to write with spaces between words?
grammar_string = r"""
root ::= sentence{1,2}
allowedchars ::= [\u4e00 - \u9faf, \u3000-\u303f, \u3040-\u309F, \u30a0-\u30ff]
word ::= (unicodechar)+
unicodechar ::= [\u0000-\uffff]
sentence ::= word particle word desu end
particle ::= "が"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], grammar=my_grammar)

{'id': 'chatcmpl-8ebe6514-5a4f-4201-9b37-c58890a0b2da',
 'object': 'chat.completion',
 'created': 1739716189,
 'model': '/data/ai_club/llms/qwen2.5-7b-instruct-q8_0-00001-of-00003.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant', 'content': '赤色は炎のように光ってます が 楽しく感じます です。'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 50, 'completion_tokens': 19, 'total_tokens': 69}}

In [6]:
LLM_GLOBAL_INSTANCE.reset()
system_input = "You are a helpful assistant. Respond with prompts suitable for Japanese children's books, following the [word] が [word] です form."
user_input = "赤色は何をしてるですか？"

In [7]:
#only 1 or 2 allowedchar makes for much shorter+concise sentences
#adding more sentences makes them incoherent
grammar_string = r"""
root ::= sentence
allowedchars ::= [\u0000-\uffff]
word ::= allowedchars
sentence ::= word particle word desu end
particle ::= "が"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

#grammar works better with Japanese prompts
#English prompts are better without grammar
#Not much variety between responses though
#How does Llama grammar work??!!? Logits?

In [8]:
training_data = list()
training_size = 10
for i in range(training_size):
    response = LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    grammar=my_grammar
    )
    training_data.append(response['choices'][0]["message"]["content"])
training_data

['赤が喜です。',
 '赤が喜です。',
 '赤が火です。',
 '赤が喜です。',
 '赤が力です。',
 '赤が力です。',
 '赤が怒です。',
 '赤が喜です。',
 '赤が怒です。',
 '赤が光です。']

In [9]:
# i and na-adj don't work well
LLM_GLOBAL_INSTANCE.reset()
system_input = "You are a helpful assistant. Respond with prompts suitable for Japanese children's books, using adjectives to describe in the [word] が [adjective] form."
user_input = "太陽の景色を描いてください"
grammar_string = r"""
root ::= sentence
allowedchars ::= [\u0000-\uffff]
word ::= allowedchars
sentence ::= word particle adjective shii end
particle ::= "が"
adjective ::= [\u0000-\uffff]
shii ::= "しい"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

In [10]:
training_data = list()
training_size = 10
for i in range(training_size):
    response = LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    grammar=my_grammar
    )
    training_data.append(response['choices'][0]["message"]["content"])
training_data

['太が出しい。',
 '太が出しい。',
 '太が出しい。',
 '太が出しい。',
 '太が輝しい。',
 '太があしい。',
 '太が出しい。',
 '太があしい。',
 '太が出しい。',
 '太が出しい。']

In [11]:
LLM_GLOBAL_INSTANCE.reset()
system_input = "You are a helpful assistant. Respond with prompts suitable for Japanese children's books, using the かどうか grammar pattern"
user_input = "Ask me a quesiton in Japanese."
grammar_string = r"""
root ::= sentence
allowedchars ::= [\u0000-\uffff]
word ::= allowedchars+
sentence ::= word particle word end
particle ::= "かどうか"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

In [13]:
training_data = list()
training_size = 5
for i in range(training_size):
    response = LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    #grammar=my_grammar
    )
    training_data.append(response['choices'][0]["message"]["content"])
training_data

['あなたは猫が好きかどうか教えてください。',
 '日本のどこに住むakahou か？（你住在日本的哪里啊？）',
 'あなたはお気に入りのキャラクターがいたとき、そのキャラクターと物語の冒険をするとかどうか？',
 '日本語で質問をしますね。日本のお子さん向けの本に使われそうな質問です。\n\nあなたはigatorかどうか知っていますか？',
 'あなたは今日学校に行きますか？']